Setting Directory

In [ ]:
import os
from pathlib import Path

In [ ]:
import logging

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
%pwd

Training Component

Import Configurations

In [ ]:
from skinCancerDiagnosis.entity.config_entity import TrailTrainingConfig
from skinCancerDiagnosis.config.configuration import ConfugarationManager
from skinCancerDiagnosis.components.data_prep import DataGenerator

Import Nesessary Libraries

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

Training Component

In [ ]:
class vgg19Training:
    def __init__(self, config:TrailTrainingConfig):
        self.config = config
        self.early_stopping = EarlyStopping(monitor='val_loss',patience=config.params_patience,restore_best_weights=True)

    def define_base_model(self):
        base_model = VGG19(weights=self.config.params_weights, include_top=self.config.params_include_top)

        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(8, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=predictions)

        for layer in base_model.layers:
            layer.trainable = False

        logging.info(model.summary())
        
        return model
    
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model_path = os.path.join(path,'vgg19.h5')
        model.save(model_path)
    
    def get_model_and_compile(self):
        with tf.device('/GPU:0'):
            self.vgg19_model = self.define_base_model()
            self.vgg19_model.compile(
                optimizer = "adam",
                loss = "categorical_crossentropy",
                metrics = ['accuracy']
            )
    
    def train_model(self,train_generator,val_generator):
        with tf.device('/GPU:0'):
            self.history = self.vgg19_model.fit(
                train_generator,
                steps_per_epoch = train_generator.samples//train_generator.batch_size,
                epochs = self.config.params_epochs,
                validation_data = val_generator,
                validation_steps = val_generator.samples//val_generator.batch_size,
                callbacks = [self.early_stopping]
            )

        self.save_model(
            path=self.config.root_dir,
            model=self.vgg19_model
        )



Training Pipeline

In [ ]:
try:
    config = ConfugarationManager()

    data_prep_config = config.get_data_prep_config()
    generator = DataGenerator(config=data_prep_config)
    train_generator = generator.get_train_generator()
    val_generator = generator.get_val_generator()
    
    trail_train_config = config.get_trail_training_config()
    vgg_trainer = vgg19Training(config=trail_train_config)
    vgg_trainer.get_model_and_compile()
    vgg_trainer.train_model(train_generator=train_generator,val_generator=val_generator)

except Exception as e:
    raise e